In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import spacy; from spacy.lang.en import English; nlp = English()
from metrics import *
from utils import *

In [2]:
data = getData()

/home/pranav_pro/SchoolSystem/models/essay/utils.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  data['illegible']=data.essay.str.contains('(\?\?\?|illegible|not legible)')


In [3]:
w2vModel = Word2Vec.load('Essayw2v.model')
vocab = set(w2vModel.wv.vocab)
dic = w2vModel.wv
w2v = dict(zip(w2vModel.wv.index2word, w2vModel.wv.syn0))

/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  after removing the cwd from sys.path.


In [4]:
def convert2tokens(essay):
    essay = basicClean(essay)
    essay = replaceNER(essay)
    essay = nlp(essay)
#     essay = [token.text for token in essay if ((not token.is_punct) and (not token.is_stop))]
    essay = [token.text for token in essay if (not token.is_punct)]
    return essay

In [5]:
tqdm.pandas()
data['tokenize'] = data['essay'].progress_apply(lambda x: convert2tokens(x))

100%|██████████| 12701/12701 [00:18<00:00, 696.30it/s]


In [6]:
data['final_score'] = data['domain1_score']
mul = [5,10,20,20,15,15,2,1]
data['final_score'] = data.apply(lambda row: row['final_score']*mul[row['essay_set']-1], axis=1)
data.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6,tokenize,final_score
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[dear, local, newspaper, i, think, effects, co...",40
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[dear, @caps, @caps, i, believe, that, using, ...",45
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[dear, @caps, @caps, @caps, more, and, more, p...",35
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[dear, local, newspaper, @caps, i, have, found...",50
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[dear, @location, i, know, having, computers, ...",40


In [7]:
tokenizer = Tokenizer(nb_words=len(vocab))
tokenizer.fit_on_texts(data['tokenize'])
data['sequence'] = tokenizer.texts_to_sequences(data['tokenize'])
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 42443 unique tokens.


In [8]:
# MaxSequence = max([len(seq) for seq in data['sequence']])
MaxSequence = 1200
print(MaxSequence)
tqdm.pandas()
inputData = pad_sequences(data['sequence'], maxlen=MaxSequence)
# data['padSequence'] = data['sequence'].progress_apply(lambda seq: pad_sequences([seq],maxlen=MaxSequence))

1200


In [9]:
data.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6,tokenize,final_score,sequence
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[dear, local, newspaper, i, think, effects, co...",40,"[302, 485, 427, 7, 67, 524, 42, 20, 18, 21, 19..."
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[dear, @caps, @caps, i, believe, that, using, ...",45,"[302, 11, 11, 7, 176, 6, 312, 42, 60, 634, 105..."
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[dear, @caps, @caps, @caps, more, and, more, p...",35,"[302, 11, 11, 11, 76, 3, 76, 21, 122, 42, 35, ..."
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[dear, local, newspaper, @caps, i, have, found...",50,"[302, 485, 427, 11, 7, 20, 250, 6, 73, 1267, 1..."
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[dear, @location, i, know, having, computers, ...",40,"[302, 127, 7, 90, 235, 42, 95, 4, 346, 288, 18..."


In [10]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if word in vocab:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = dic[word]

In [11]:
embedding_layer = layers.Embedding(len(word_index) + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=MaxSequence,
                            trainable=False)

In [12]:
embedding_layer

In [27]:
def buildData(x,y):  
    y = y/6
    # split the data into a training set and a validation set
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    x = x[indices]
    y = y[indices]
    nb_validation_samples = int(0.1 * x.shape[0])

    trainX = x[:-nb_validation_samples]
    trainY = y[:-nb_validation_samples]
    testX = x[-nb_validation_samples:]
    testY = y[-nb_validation_samples:]
    return trainX, trainY, testX, testY

In [56]:
def build_model():
    model = Sequential()
    model.add(layers.InputLayer((MaxSequence,), dtype='int32'))
    model.add(embedding_layer)
    model.add(layers.Conv1D(64, 5, activation='relu'))
    model.add(layers.MaxPooling1D(35))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1,activation="linear"))

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mean_squared_error',
                optimizer='adam',
                metrics=['mean_squared_error'])
    return model

In [57]:
tf.test.is_gpu_available()

True

In [58]:
from tensorflow.keras.callbacks import Callback

class IntervalEvaluation(Callback):
    def __init__(self, validation_data=(), interval=10):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict_proba(self.X_val, verbose=0)
            score = kappa(self.y_val, y_pred,weights='quadratic')
            print("interval evaluation - epoch: {:d} - score: {:.6f}".format(epoch, score))

In [59]:
trainX, trainY, testX, testY = buildData(inputData,np.array(data['final_score'].values.tolist()))
model = build_model()
model.summary()
ival = IntervalEvaluation(validation_data=(testX, testY), interval=1)
# history = model.fit(trainX, trainY, epochs=5,  callbacks=[ival], validation_split = 0.1, verbose=1)
history = model.fit(trainX, trainY, epochs=10,  callbacks=[ival], verbose=1)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1200, 300)         12733200  
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1196, 64)          96064     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 34, 64)            0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 2176)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2176)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)                139328    
___________________________

In [129]:
model.save("EssayModelRNN.h5")